<a href="https://colab.research.google.com/github/quangtrung0220/SieuAmTim/blob/master/SieuAmTim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### ***Import các thư viện cần thiết***

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader

from torchvision import transforms
import torchvision

import matplotlib.pyplot as plt

from collections import namedtuple

from sklearn.metrics import classification_report

import cv2

import numpy as np


### ***Clone git để lấy dữ liệu ảnh train và test***

In [ ]:
# Mount với drive để lấy datasets 
# from google.colab import drive
# drive.mount('/content/drive')

# Link git lấy dataset 
!git clone https://github.com/quangtrung0220/SieuAmTim.git
traindir = "/content/SieuAmTim/DATA_CHAMBER_2021/train"
testdir = "/content/SieuAmTim/DATA_CHAMBER_2021/test"


Cloning into 'SieuAmTim'...
remote: Enumerating objects: 8346, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 8346 (delta 4), reused 17 (delta 3), pack-reused 8328
Receiving objects: 100% (8346/8346), 488.04 MiB | 26.75 MiB/s, done.
Resolving deltas: 100% (6/6), done.
Checking out files: 100% (8328/8328), done.


### ***Lấy ra các nhãn - classes và chuẩn bị dữ liệu***


---



*   Các nhãn: 2C, 3C, 4C
*   Các loại kích cỡ ảnh train và test: 224x224, 32x32
*   Preprocess: Cân bằng sáng: histogram, Lọc nhiễu: GaussianBlur
*   Data Augmentation: RandomCrop, RandomHorizontalFlip, RandomVerticalFlip
*   Đưa vào batch để tận dụng khả năng xử lý song song của GPU
*   Folder train và test lần lượt trong: traindir và testdir






In [ ]:
# Hàm lấy ra các classes
def get_clases():
  classes = ['2C', '3C', '4C']
  return classes

TrainTest = namedtuple('TrainTest', ['train', 'test'])

# Hàm chuẩn bị dữ liệu
def prepare_data():

  # Preprocess
  gaussianBlur = transforms.GaussianBlur(3)
  histogramEqualize = transforms.RandomEqualize(p=0.5)

  # #Normalize
  # norm = transforms.Normalize(mean=[0.485, 0.456, 0.406],
  #      std=[0.229, 0.224, 0.225])

  #Augmentation for all
  horizontal = transforms.RandomHorizontalFlip()
  vertical = transforms.RandomVerticalFlip()

  #Image 224x224
  resize_224 = transforms.Resize((224,224))
  crop_224 = transforms.RandomCrop(224, padding=4)
  
  #Image 32x32
  resize_32 = transforms.Resize((32,32))
  crop_32 = transforms.RandomCrop(32, padding=4)

  #To tensor
  tensor = transforms.ToTensor()

  #Raw image
  transform_train_raw224 = transforms.Compose([resize_224, tensor])
  transform_train_raw232 = transforms.Compose([resize_32, tensor])

  #Augmentation image
  transform_train_aug224 = transforms.Compose([resize_224, crop_224, horizontal, vertical, tensor])
  transform_train_aug32 = transforms.Compose([resize_32, crop_32, horizontal, vertical, tensor])

  #Preprocess image
  transform_train_pre224 = transforms.Compose([resize_224, gaussianBlur, histogramEqualize, tensor])
  transform_train_pre32 = transforms.Compose([resize_32, gaussianBlur, histogramEqualize, tensor])

  #Test image
  transform_test_224 = transforms.Compose([resize_224, tensor])
  transform_test_32 = transforms.Compose([resize_32, tensor])

  # transform_test = transforms.Compose([
  #   transforms.Resize((224,224)),
  #   # transforms.Resize((32,32)),

  #   # Preprocess
  #   transforms.GaussianBlur(3),
  #   transforms.RandomEqualize(p=0.5),

  #   transforms.ToTensor(),
  #   transforms.Normalize(mean=[0.485, 0.456, 0.406],
  #      std=[0.229, 0.224, 0.225])
  # ])
  trainset = torchvision.datasets.ImageFolder(root=traindir, transform=transform_train_aug224)
  testset = torchvision.datasets.ImageFolder(root=testdir, transform=transform_test_224)
  return TrainTest(train=trainset, test=testset)

# Hàm load dữ liệu
def prepare_loader(datasets):
  trainloader = DataLoader(dataset=datasets.train, batch_size=35, shuffle=True, num_workers=4)
  testloader = DataLoader(dataset=datasets.test, batch_size=35, shuffle=False, num_workers=4)
  return TrainTest(train=trainloader, test=testloader)

### ***Hàm train và test***


---



*   BatchSize = 35
*   Report sau mỗi 20 epoch 
*   Train: cho đi qua model và tính lỗi bằng loss_func sau đó cập nhật tham số mới
*   Test: khai báo chế độ eval(đánh giá - evaluate) và trả về nhãn dự đoán và nhãn thực tếtế



In [ ]:
# Data = prepare_data()
# img, label = Data.train[3000]
# print(label)
# plt.show(img)
# print(img)
# s = Data.train.imgs[0]
# print(s)

In [ ]:
# Hàm train 
def train_epoch(epoch, model, loader, loss_func, optimizer, device):
  true_result = []
  pred_result = []

  # gọi hàm train để model biết mình phải cần huấn luyện 
  model.train()
  running_loss = 0.0
  reporting_steps = 20
  for i, (images, labels) in enumerate(loader):
    images, labels = images.to(device), labels.to(device)
    outputs = model(images)
    true_result += list(labels.cpu().numpy())
    _, predicted = torch.max(outputs, dim=1)
    pred_result += list(predicted.cpu().numpy())
    # Tính lỗi 
    loss = loss_func(outputs, labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
    if i % reporting_steps == reporting_steps-1:
      print(f"Epoch {epoch} step {i} ave_loss {running_loss/reporting_steps:.4f}")
      running_loss = 0.0
  # return pred_result, true_result

# hàm test 
def test_epoch(epoch, model, loader, device):
  ytrue = []
  ypred = []
  # Gọi hàm eval để model biết đang ở chế độ test 
  with torch.no_grad():
    model.eval()
    
    for i, (images, labels) in enumerate(loader):
      images, labels = images.to(device), labels.to(device)
      outputs = model(images)
      _, predicted = torch.max(outputs, dim=1)

      ytrue += list(labels.cpu().numpy())
      ypred += list(predicted.cpu().numpy())

  return ypred, ytrue

### ***Hàm main***

---


*   model_in = 'vgg16', 'resnet50' hoặc 'desnet': tên mạng Cnn đùng để huấn luyện
*   PATH: đường dẫn lưu trọng số sau khi huấn luyện
*   Đặc trưng ra = 3 vì có 3 lớp 2C, 3C, 4C
*   Hàm lỗi được sử dụng: Cross-Entropy
*   Hàm tối ưu đước sử dụng: SGD - Stochastic Gradient Descent




In [ ]:
def main(PATH='./model.pth', model_in=''):
  classes = get_clases()
  datasets = prepare_data()
  # img, label = datasets.train[0]
  # plt.imshow(img)
  # print(classes[label], img.size)
  # print('train', len(datasets.train), 'test', len(datasets.test))
  
  loaders = prepare_loader(datasets)

  # images, labels = iter(loaders.train).next()
  # print(images.shape, labels.shape)

  device = torch.device("cuda:0")
  torch.cuda.empty_cache()

  # model = ResNet50().to(device)
  # model = VGG16().to(device)
  # images, labels = iter(loaders.train).next()
  # outputs = model(images)
  # print(outputs.shape)
  # print(outputs[0])
  # _, predicted = torch.max(outputs, dim=1)
  # print(predicted)
  # imshow(images, labels, predicted, classes)

# Load model 
  if model_in == 'vgg16':
    model = torchvision.models.vgg16()
    model.classifier[6] = torch.nn.modules.linear.Linear(in_features=4096, out_features=3, bias=True)
    # if torch.cuda.is_available():
    model.to(device=device)
  elif model_in == 'resnet50':
    model = torchvision.models.resnet50(pretrained=False, progress=False)
    model.fc = torch.nn.modules.linear.Linear(in_features=2048, out_features=3, bias=True)  
    # if torch.cuda.is_available():
    model.to(device=device)
  elif model_in == 'desnet':
    model = torchvision.models.densenet121(pretrained=False, progress=False)
    model.classifier = torch.nn.modules.linear.Linear(in_features=1024, out_features=3, bias=True)
    # if torch.cuda.is_available():
    model.to(device=device)
  else:
    pass  

  # array_accuracy = []

  loss_func = nn.CrossEntropyLoss()
  optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)
  for epoch in range(10):
    print(f'Epoch {epoch} report: ')
    
    # pred_result, true_result = 
    train_epoch(epoch, model, loaders.train, loss_func, optimizer, device)
    # print('Train report: ')
    # print(classification_report(true_result, pred_result, target_names=classes))

    print('Test report: ')
    ypred, ytrue = test_epoch(epoch, model, loaders.test, device)

    # accuracy = (ytrue_test==ypred_test).sum() / len(ytrue_test)
    # array_accuracy.append(accuracy)

    print(classification_report(ytrue, ypred, target_names=classes))

    torch.save(model.state_dict(), PATH)

  return model

model = main(PATH="./vgg16.pth", model_in='vgg16')

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch 0 report: 
Epoch 0 step 19 ave_loss 1.0997
Epoch 0 step 39 ave_loss 1.0959
Epoch 0 step 59 ave_loss 1.0969
Epoch 0 step 79 ave_loss 1.0857
Epoch 0 step 99 ave_loss 1.0824
Epoch 0 step 119 ave_loss 1.0837
Epoch 0 step 139 ave_loss 1.0739
Epoch 0 step 159 ave_loss 1.0651
Epoch 0 step 179 ave_loss 1.0420
Test report: 
              precision    recall  f1-score   support

          2C       0.38      0.88      0.53       409
          3C       1.00      0.04      0.07       367
          4C       0.69      0.53      0.60       831

    accuracy                           0.51      1607
   macro avg       0.69      0.48      0.40      1607
weighted avg       0.68      0.51      0.46      1607

Epoch 1 report: 


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch 1 step 19 ave_loss 1.0015
Epoch 1 step 39 ave_loss 0.9567
Epoch 1 step 59 ave_loss 0.9557
Epoch 1 step 79 ave_loss 0.9997
Epoch 1 step 99 ave_loss 0.9371
Epoch 1 step 119 ave_loss 0.8417
Epoch 1 step 139 ave_loss 0.8930
Epoch 1 step 159 ave_loss 0.7228
Epoch 1 step 179 ave_loss 0.7327
Test report: 
              precision    recall  f1-score   support

          2C       0.48      1.00      0.65       409
          3C       0.70      0.29      0.41       367
          4C       0.95      0.70      0.81       831

    accuracy                           0.68      1607
   macro avg       0.71      0.66      0.62      1607
weighted avg       0.78      0.68      0.68      1607

Epoch 2 report: 


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


...

Epoch 7 step 19 ave_loss 0.0521
Epoch 7 step 39 ave_loss 0.0114
Epoch 7 step 59 ave_loss 0.0555
Epoch 7 step 79 ave_loss 0.0703
Epoch 7 step 99 ave_loss 0.0415
Epoch 7 step 119 ave_loss 0.0298
Epoch 7 step 139 ave_loss 0.0406
Epoch 7 step 159 ave_loss 0.0402
Epoch 7 step 179 ave_loss 0.0293
Test report: 
              precision    recall  f1-score   support

          2C       0.55      0.98      0.70       409
          3C       0.76      0.77      0.76       367
          4C       0.98      0.60      0.75       831

    accuracy                           0.73      1607
   macro avg       0.76      0.78      0.74      1607
weighted avg       0.82      0.73      0.74      1607

Epoch 8 report: 


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch 8 step 19 ave_loss 0.0205
Epoch 8 step 39 ave_loss 0.0619
Epoch 8 step 59 ave_loss 0.0421
Epoch 8 step 79 ave_loss 0.0294
Epoch 8 step 99 ave_loss 0.0164
Epoch 8 step 119 ave_loss 0.0342
Epoch 8 step 139 ave_loss 0.0479
Epoch 8 step 159 ave_loss 0.0332
Epoch 8 step 179 ave_loss 0.0169
Test report: 
              precision    recall  f1-score   support

          2C       0.74      0.72      0.73       409
          3C       0.65      0.92      0.76       367
          4C       0.94      0.78      0.86       831

    accuracy                           0.80      1607
   macro avg       0.78      0.81      0.78      1607
weighted avg       0.82      0.80      0.80      1607

Epoch 9 report: 


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch 9 step 19 ave_loss 0.0171
Epoch 9 step 39 ave_loss 0.0124
Epoch 9 step 59 ave_loss 0.0359
Epoch 9 step 79 ave_loss 0.0485
Epoch 9 step 99 ave_loss 0.0284
Epoch 9 step 119 ave_loss 0.0393
Epoch 9 step 139 ave_loss 0.0357
Epoch 9 step 159 ave_loss 0.0190
Epoch 9 step 179 ave_loss 0.0080
Test report: 
              precision    recall  f1-score   support

          2C       0.64      0.98      0.77       409
          3C       0.66      0.80      0.72       367
          4C       0.97      0.63      0.76       831

    accuracy                           0.76      1607
   macro avg       0.76      0.80      0.75      1607
weighted avg       0.82      0.76      0.76      1607



In [ ]:
# Kết quả sau 10 epoch:

# Train_Accuracy:
# VGG16_Ảnh resize 224: [35, 38, 38, 41, 54, 70, 82, 89, 93, 95]
# VGG16_Ảnh resize 32: [36, 42, 54, 65, 81, 89, 92, 94, 97, 97]
# ResNet50_Ảnh resize 224: [35, 38, 38, 41, 54, 70, 82, 89, 93, 95]
# Desnet121_Ảnh resize 224: [68, 93, 97, 99, 99, 99, 100, 100, 100, 100]
# Desnet121_Ảnh resize 32: [63, 80, 91, 94, 97, 97, 97, 98, 98, 99]

# Test_Accuracy
# VGG16_Ảnh resize 224: [32, 25, 27, 48, 66, 62, 92, 90, 85, 87]
# VGG16_Ảnh resize 32: [23, 56, 63, 74, 82, 83, 87, 85, 91, 83]
# ResNet50_Ảnh resize 224: [35, 38, 38, 41, 54, 70, 82, 89, 93, 95]
# Desnet121_Ảnh resize 224: [66, 71, 78, 74, 74, 72, 82, 87, 87, 88]
# Desnet121_Ảnh resize 32: [78, 83, 79, 78, 77, 72, 70, 78, 78, 82]